<br>
<img src="images/recgov_booking.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov
<br>
<img src="images/recgov_tou.png" width=80% style="display: inline-block">
<br>
image source: http://www.recreation.gov/marketing.do?goto=/acm/shareData.htm

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyvirtualdisplay import Display
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime
from datetime import timedelta, date
import sys
import requests

In [13]:
print('starting display')
display = Display(visible=0, size=(1024, 768))
display.start()

print('setting up web browser')
browser = webdriver.Firefox()

starting display
setting up web browser


In [20]:
def check_recreationgov(facility_url, start_date, stay_length) :
    
    browser.set_window_size(1366, 768)
    
    #browser = mechanize.Browser()
    #user_agent_str = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
    #browser.addheaders = [('User-agent', user_agent_str)]	
    end_date = "{:%m/%d/%Y}".format(datetime.datetime.strptime(start_date, "%m/%d/%Y") + timedelta(1))

    print('getting url')
    try:
        browser.get(facility_url)
        browser.set_script_timeout(30)
        browser.set_page_load_timeout(30) # seconds

        #print(browser.page_source)
        #browser.open(facility_url)
    except Exception as ex:
        print("Unable to open url: " + facility_url)
        print(ex)
        return pd.DataFrame()
    
    print('browsing form')
    form = browser.find_element_by_name('unifSearchForm')
    arrival = form.find_element_by_name('arrivalDate')
    departure = form.find_element_by_name('departureDate')

    arrival.send_keys(start_date)
    departure.send_keys(end_date)
    form.submit()
    soup = BeautifulSoup(browser.page_source, 'lxml')

    divs = soup.findAll('div', attrs={"class" : 'matchSummary'})
    query_result = divs[0].text
    results = pd.DataFrame()

    for filter_name in soup('div', {"class": re.compile("searchTypeFilter.*")}):
        parts = filter_name.text.split('(')
        entry_name = parts[0]

        name = entry_name.lower().strip().replace(' ', '_')
        name = name.replace("-","")

        # some names have spaces around the '-', some dont
        # i.e. standard - full vs standard-full
        # remove __ to avoid accidentally concatenating standardfull if there arent spaces
        # around the -
        name = name.replace("__", "_")
        results[name] = [parts[1].split(')')[0]]

    results = results.drop('all', axis=1)
    results['last_updated'] = ["{:%m/%d/%Y}".format(datetime.date.today())]
    results['start_date'] = start_date
    results['end_date'] = end_date

    availability_info = query_result.split(' ')
    sites_available = availability_info[0]
    #total_sites = availability_info[5]

    results['sites_available'] = [sites_available]
    #results['total_sites'] = [total_sites]
    


    return results

In [21]:
test = check_recreationgov('http://www.recreation.gov/camping/Slide-Creek/r/campgroundDetails.do?contractCode=NRSO&parkId=72370', '05/25/2016', 2)

getting url
browsing form


In [22]:
test

,standard_nonelectric,last_updated,start_date,end_date,sites_available
0,8,05/18/2016,05/25/2016,05/26/2016,8


In [23]:
    browser.close()
    display.stop()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1005'] oserror=None returncode=0 stdout="" stderr="Initializing built-in extension Generic Event Extension
Initializing built-in extension SHAPE
Initializing built-in extension MIT-SHM
Initializing built-in extension XInputExtension
Initializing built-in extension XTEST
Initializing built-in extension BIG-REQUESTS
Initializing built-in extension SYNC
Initializing built-in extension XKEYBOARD
Initializing built-in extension XC-MISC
Initializing built-in extension SECURITY
Initializing built-in extension XINERAMA
Initializing built-in extension XFIXES
Initializing built-in extension RENDER
Initializing built-in extension RANDR
Initializing built-in extension COMPOSITE
Initializing built-in extension DAMAGE
Initializing built-in extension MIT-SCREEN-SAVER
Initializing built-in extension DOUBLE-BUFFER
Initializing built-in e